In [1]:
"""Mini projet sur les menus des cantines de Rennes métropole"""
################################################################################
# fichier  : RakotosonLoic.py
# Auteur : Rakotoson Loïc
################################################################################

################################################################################
# Importation de fonctions externes :
import pickle
import csv
from datetime import datetime
from fuzzywuzzy import fuzz, process

################################################################################
# Definition locale de fonctions :

In [2]:
# Récupération des variables
def readData(nomFichier):
    listeData = []
    try:
        with open(nomFichier, 'rb') as objet_fichier:
            objet_fichier_depickler = pickle.Unpickler(objet_fichier)
            while 1:
                try:
                    listeData.append(objet_fichier_depickler.load())
                except EOFError:
                    break
    except (IOError, OSError):
        print("Problème à l'ouverture du fichier")
    return listeData

In [3]:
# Afficher joliment le menu pour un age et une date précis
def affiche_menu(date, age, data):
    for ligne in data[1:]:
        if ligne[0] == age and ligne[1] == datetime.strptime(
                date, '%Y-%m-%d').date():
            print(
                "Age: {}\nDate: {}\nEntrée: {}\nPlat: {}\nLégume: {}\nLaitage: {}\nDéssert: {}\nGoûter :{}\n"
                .format(age, date, ligne[3], ligne[5], ligne[7], ligne[8],
                        ligne[10], ligne[11]))

In [4]:
# Extraire une colonne
def colonne_extract(col, data):
    if isinstance(col, str):  # Si le nom est donné
        col = data[0].index(col)  # Changer en numéro de colonne

    colonne = [ligne[col] for ligne in data[1:]]
    return colonne

In [5]:
# Dictionnaire des mots semblables
def mot_semblable(col, data, rate=91):
    colonne = colonne_extract(col, data)
    lensemble = list(set(colonne))
    dico = {}
    index = 1
    for elt1 in lensemble[1:]:
        index += 1
        for elt2 in lensemble[index:]:
            if fuzz.ratio(elt1, elt2) >= rate:
                elt1, elt2 = min([elt1, elt2],
                                 key=colonne.count), max([elt1, elt2],
                                                         key=colonne.count)
                dico[elt1] = elt2
    return dico

In [6]:
# Extraire menu en fonction de l'age
def menu_age(age, data):
    donnees = [ligne for ligne in data[1:] if ligne[0] == age]
    donnees.insert(0, data[0]) # entête
    return donnees

In [7]:
# Extraire menu entre deux dates
def menu_date(debut, fin, data):
    debut = datetime.strptime(debut, '%Y-%m-%d').date()
    fin = datetime.strptime(fin, '%Y-%m-%d').date()
    donnees = [ligne for ligne in data[1:] if debut <= ligne[1] <= fin]
    donnees.insert(0, data[0]) #entête
    return donnees

In [8]:
# Tri en fonction de la date
def menu_datetri(data):
    trie = sorted(data[1:], key=lambda ligne:ligne[1])
    trie.insert(0, data[0]) #entête
    return trie

In [9]:
# Calculer et d’afficher la fréquence d’apparition des produitspour une colonne
def frequence(col, data):
    colonne = colonne_extract(col, data)
    menu = set(colonne)
    print("Fréquence de {}(s): ({} sur la période)\n".format(col, len(menu)))
    total, freq = 0, 0
    for element in menu:
        freq = colonne.count(element) * 100 / len(colonne)
        total += freq
        print("{}: {:.2f} %".format(element, freq))
    print("\nTotal: {} %".format(total))

In [10]:
################################################################################
# Definition de classe

################################################################################
#Récupération des données

listeData = readData('data.pic')
#Les variables ci-dessous correspondent aux réponses (exemple aide_ex_11 => exercice 1 question 1)
aide_ex_11 = listeData[0]
aide_ex_12 = listeData[1]
aide_ex_21 = listeData[2]
aide_ex_22 = listeData[3]
aide_ex_31 = listeData[4]
aide_ex_32 = listeData[5]
aide_ex_33 = listeData[6]
aide_ex_42 = listeData[7]

################################################################################
# Corps principal du programme :
# Constantes
entrees_crudites = ('Pamplemousse', 'Carottes râpées', 'Radis',
                    'Champignons et filet de citron', 'Tomates à la menthe',
                    'Concombre', 'Concombre à la grècque', 'Melon',
                    'Tomates à la ciboulette', 'Tomates au basilic',
                    'Carottes à la martiniquaise', 'Champignons',
                    'Salade de carottes et pommes', 'Champignons citronnés',
                    'Pastèque', 'Betteraves crues râpées', "Mousse d'avocats",
                    "Salade d'avocats et tomates", 'Carottes, pommes, gruyère',
                    'Salade de tomates', 'Carottes râpées et gruyère')

desserts_fruits_crus = ('Banane', 'Clémentine', 'Orange',
                        'Corbeille de fruits', 'Kiwi', 'Banane mixée', 'Poire',
                        'Pomme', 'Fraises', 'Banane et framboise mixée',
                        'Nectarine', 'Pêche', 'Abricot', 'Prune', 'Ananas',
                        'Banane et fraise mixée', 'FLS', 'Melon', 'Raisin',
                        'Poire mixée', 'Fruits mixés')

plats_acides_gras = ('Poisson sauce tomate fraîche', 'Gratin de pâtes au thon',
                     'Thon', 'Sardine', 'Poisson', 'Poisson sauce tomate',
                     'Poisson au citron', 'Poisson beurre citron')

# Les données
## Exercice 1: Importation
Lire les données présentes dans le fichier portant l’extension « data.csv »

In [11]:
with open("data.csv", 'r', encoding="utf-8") as file:
    data = [ligne for ligne in csv.reader(file, delimiter=";")]

Transformer la date en type « DateTime »

In [12]:
for ligne in data[1:]:
    ligne[1] = datetime.strptime(ligne[1], '%Y-%m-%d').date()

Écrire une fonction qui permet d’afficher (joliment) le menu pour une date et un âge (petits,moyens, grands) précis.
*On considère qu'il est possible d'avoir plusieurs menu pour une même date et même age*

In [13]:
affiche_menu('2017-01-18', 'petits', data)

Age: petits
Date: 2017-01-18
Entrée: 
Plat: Jambon
Légume: Purée de poireauxx et pommes de terre
Laitage: 
Déssert: Compote de fruits
Goûter :Crème au chocolat/ pain/ eau



## Exercice 2 : Nettoyage des données
À partir du résultat précédent, créer une fonction qui renvoie un dictionnaire associant les orthographes « considérées » comme semblable.

In [14]:
mot_semblable('dessert', data)

{'Compote poires et pêches': 'Compote poires et pêches',
 'Banane et framboise mixée': 'Banane et fraise mixée',
 'Compote pommes et abricots': 'Compote pomme et abricot',
 'Compote pommes et bananes': 'Compote pommes/ bananes',
 'Compote de pêches': 'Compotée de pêches',
 'Compote pomme/ framboise': 'Compote pomme/ framboise',
 'Compotée de nectarine': 'Compote de nectarine',
 'Compote pommes et poires': 'Compote pommes/ poires',
 'Compote de pommes et abricots': 'Compote pomme et abricot',
 'Compote de pommes et nectarines': 'Compote de pommes et nectarines',
 'Compote pommes nectarines': 'Compote pommes nectarines',
 'Compote de pommes et pêches au sirop': 'Compote de pommes et pêche au sirop',
 'Compote pomme/ poire': 'Compote pommes/ poires'}

Appliquer les fonctions précédentes à l’ensemble des colonnes et uniformiser l’orthographe des données

In [15]:
for ligne in data[1:]:
    for col in (3, 4, 5, 7, 8, 10, 11):
        for ancien, nouveau in mot_semblable(col, data).items():
            if ancien in ligne:
                ligne[ligne.index(ancien)] = nouveau

# Traitement des données
## Exercice 3 : Sélection et tri des données
Écrire une fonction qui permet d’extraire uniquement les menus pour les grands, moyens ou petits.

In [16]:
menu_age('petits', data)[:3]

[['age',
  'date',
  'picto entrée',
  'entrée',
  'picto plat',
  'plat',
  'picto légumes',
  'légume',
  'laitage',
  'picto dessert',
  'dessert',
  'goûter'],
 ['petits',
  datetime.date(2017, 1, 2),
  '',
  '',
  '',
  'Thon',
  '',
  'Purée rose',
  '',
  'FLS',
  'Compote de pommes',
  'Compote/ pain/ lait de croissance'],
 ['petits',
  datetime.date(2017, 1, 12),
  '',
  '',
  '',
  'Escalope de poulet',
  '',
  'Purée de courgettes et pommes de terre',
  '',
  'FLS',
  'Compote de poires',
  'Semoule au lait/ eau']]

Ecrire une fonction qui permet d’extraire uniquement les menus compris entre deux dates

In [17]:
menu_date("2017-01-05", "2017-01-05", data)

[['age',
  'date',
  'picto entrée',
  'entrée',
  'picto plat',
  'plat',
  'picto légumes',
  'légume',
  'laitage',
  'picto dessert',
  'dessert',
  'goûter'],
 ['moyens',
  datetime.date(2017, 1, 5),
  '',
  "Pointes d'asperges",
  '',
  'Gnocchis',
  '',
  '',
  '',
  'FLS',
  'Clémentine',
  'Pain/ fromage/ eau'],
 ['grands',
  datetime.date(2017, 1, 5),
  '',
  "Pointes d'asperges",
  '',
  'Gnocchis',
  '',
  'Salade en chiffonnade',
  '',
  'FLS',
  'Clémentine',
  'Pain/ fromage/ eau'],
 ['petits',
  datetime.date(2017, 1, 5),
  '',
  '',
  '',
  'Gnocchis',
  '',
  'Purée framboisine',
  '',
  '',
  'Compote de fruits',
  'Crème impériale/ eau']]

Écrire une fonction qui permet de trier les repas en fonction de la date. La fonction prendra en paramètre une liste de repas et renverra la liste triée.

In [18]:
menu = menu_date("2017-01-04", "2017-01-07", menu_age('petits', data))
menu_datetri(menu)

[['age',
  'date',
  'picto entrée',
  'entrée',
  'picto plat',
  'plat',
  'picto légumes',
  'légume',
  'laitage',
  'picto dessert',
  'dessert',
  'goûter'],
 ['petits',
  datetime.date(2017, 1, 4),
  '',
  '',
  'VBF',
  'Steak',
  '',
  'Purée de pommes de terre, poireaux, carottes',
  '',
  'FLS',
  'Banane mixée',
  'Compote/ pain/ lait de croissance'],
 ['petits',
  datetime.date(2017, 1, 5),
  '',
  '',
  '',
  'Gnocchis',
  '',
  'Purée framboisine',
  '',
  '',
  'Compote de fruits',
  'Crème impériale/ eau'],
 ['petits',
  datetime.date(2017, 1, 6),
  '',
  '',
  'VPF',
  'Rôti de porc',
  '',
  'Purée rose',
  '',
  'FLS',
  'Compote de poires',
  'Yaourt nature/ boudoirs/ eau']]

## Exercice 4: Résumé des données
Écrire une fonction qui permet de calculer et d’afficher la fréquence d’apparition des produits pour une colonne particulière

In [19]:
frequence("plat", data)

Fréquence de plat(s): (23 sur la période)

Rôti de porc: 12.47 %
Potée (rôti de porc): 0.39 %
Gratin de pâtes au thon: 0.92 %
Rôti de bœuf: 15.88 %
Steak: 6.69 %
Gigot d'agneau: 1.84 %
Quiche jambon/ quiche nature: 0.26 %
Gnocchis: 3.02 %
Escalope de veau: 2.23 %
Poisson au citron: 1.05 %
Sardine: 0.79 %
Poisson sauce tomate fraîche: 0.92 %
Escalope de dinde: 6.17 %
Betteraves cuites: 0.13 %
Potée: 0.13 %
Omelette: 3.15 %
Poisson: 20.21 %
Poisson beurre citron: 3.54 %
Poisson sauce tomate: 0.52 %
Jambon: 6.30 %
Escalope de poulet: 10.76 %
Thon: 2.10 %
Quiche au jambon et quiche nature: 0.52 %

Total: 99.99999999999999 %


Écrire une fonction qui retourne toutes les dates de service d’un produit pour une colonne particulière

In [20]:
# dates de service d’un produit pour une colonne particulière
def produit_service(col, data):
    colonne = [[date, element] for date, element in zip(
        colonne_extract(col, data), colonne_extract(1, data))]
    dico = {}
    for element in colonne:
        if element[0] in dico:
            dico[element[0]].append(element[1].strftime("%d/%m/%Y"))
        else:
            dico[element[0]] = [element[1].strftime("%d/%m/%Y")]
    return dico